In [ ]:
! pip install unidecode

In [ ]:
import pandas as pd
import spacy
import nltk
import re
from nltk.stem import RSLPStemmer
from unidecode import unidecode

In [ ]:
!python -m spacy download pt_core_news_sm

nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')
nltk.download('rslp')

# Treino

## Análise exploratória dos dados

In [ ]:
df = pd.read_csv("../input/5cd-classificacao-emocao/6 - base_dados_treinamento - Kaggle.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
train.info()

## Pré - processamento

In [ ]:
df["sentiment_int"] = df["Label"].map({"positivo": 1, "negativo": 0})

Lowercase e acenteuação

In [ ]:
df['Texto'] = df.Texto.apply(str.lower)
df['Texto'] = df.Texto.apply(unidecode)

Retindo caracteres especiais

In [ ]:
# Substitui nickname por espaço

df = df.replace(to_replace = '\@(\w{4,15})', value = ' ', regex = True)

In [ ]:
# Substitui link por espaço

df = df.replace(to_replace ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', value = ' ', regex = True)

In [ ]:
# Substitui por espaço todos caracteres que não forem de a a Z, dígitos de 0 a 9 e o caractere sublinhado _

df['Texto'] = df['Texto'].apply(lambda x : re.sub('\W', ' ', str(x)))
df['Label'] = df['Label'].apply(lambda x : re.sub('\W', ' ', str(x)))

In [ ]:
# Substitui o caracter _ por espaço

df = df.replace(to_replace ='_', value = ' ', regex = True)

Stop word, lematização e tokenização

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
# Stop words

stop = stopwords.words('portuguese')

df['Texto'] = df['Texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
# Lematização

df['Texto'] = df['Texto'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

# Teste

## Análise exploratória dos dados

In [ ]:
df_test = pd.read_csv("../input/5cd-classificacao-emocao/6 - base_dados_teste_certificacao - Kaggle.csv", error_bad_lines=False)

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.isnull().sum()

## Pré - processamento

Lowercase e acenteuação

In [ ]:
df_test['Texto'] = df_test.Texto.apply(str.lower)
df_test['Texto'] = df_test.Texto.apply(unidecode)

Retindo caracteres especiais

In [ ]:
# Substitui nickname por espaço

df_test = df_test.replace(to_replace = '\@(\w{4,15})', value = ' ', regex = True)

In [ ]:
# Substitui link por espaço

df_test = df_test.replace(to_replace ='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', value = ' ', regex = True)

In [ ]:
# Substitui por espaço todos caracteres que não forem de a a Z, dígitos de 0 a 9 e o caractere sublinhado _

df_test['Texto'] = df_test['Texto'].apply(lambda x : re.sub('\W', ' ', str(x)))

In [ ]:
# Substitui o caracter _ por espaço

df_test = df_test.replace(to_replace ='_', value = ' ', regex = True)

Stop word, lematização e tokenização

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
# Stop words

stop = stopwords.words('portuguese')

df_test['Texto'] = df_test['Texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
# Lematização

df_test['Texto'] = df_test['Texto'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

## Modelo

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, metrics
from sklearn.feature_extraction.text import CountVectorizer

Treino e validação - Hould out

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(df["Texto"], df["sentiment_int"])

CountVectorizer para contar as palavras em cada tweet - frequencia positiva e negativa das palavras

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

In [ ]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [ ]:
x_train_count.toarray()

Regressão logística

In [ ]:
log = linear_model.LogisticRegression()
log_model = log.fit(x_train_count, train_y)

F1 score

In [ ]:
scores = model_selection.cross_val_score(log_model, x_test_count, test_y, cv = 3, scoring="f1")
print(scores)

Teste

In [ ]:
test_vect = vectorizer.transform(df_test.Texto)
test_vect = test_vect.toarray()

Predições

In [ ]:
predictions = log_model.predict(test_vect)
predictions

In [ ]:
predictions = pd.DataFrame(predictions)

In [ ]:
predictions["sentimento"] = predictions[0].map({1 : "positivo", 0 : "negativo"})

In [ ]:
predictions.insert(1, "id", range(1, 197), True)

In [ ]:
predictions.drop([0], axis=1, inplace=True)